# Docker Swarm with Blinkt!

## Init Docker Swarm on Cluster

Zunächst die IP-Adresse feststellen.

In [1]:
ip a s eth0

2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether b8:27:eb:27:c2:35 brd ff:ff:ff:ff:ff:ff
    inet 172.24.2.1/24 brd 172.24.2.255 scope global eth0:1
       valid_lft forever preferred_lft forever
    inet6 fe80::ba27:ebff:fe27:c235/64 scope link 
       valid_lft forever preferred_lft forever


In [2]:
ip a s eth0:1 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1

172.24.2.1


In [3]:
IP_LEADER=$(ip a s eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1)
echo $IP_LEADER

172.24.2.1


In [4]:
IP_LEADER_WLAN=$(ip a s wlan0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1)
echo $IP_LEADER_WLAN

172.24.1.1


In [5]:
docker node ls || (docker swarm init --advertise-addr ${IP_LEADER} && docker node ls)

Error response from daemon: This node is not a swarm manager. Use "docker swarm init" or "docker swarm join" to connect this node to swarm and try again.
Swarm initialized: current node (dchzaekl2ka9up3i2fez7lmiu) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-55azwsd7ec2mklm750dd16kzkf5thkhqfc2mwdc6u3pq23ep0q-br1gct0pzrhnfj34fuxmvga26 172.24.2.1:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
dchzaekl2ka9up3i2fez7lmiu *   cluster-00          Ready               Active              Leader


In [6]:
JOIN_TOKEN=$(docker swarm join-token -q worker)
echo $JOIN_TOKEN

SWMTKN-1-55azwsd7ec2mklm750dd16kzkf5thkhqfc2mwdc6u3pq23ep0q-br1gct0pzrhnfj34fuxmvga26


## SSH einrichten

In [7]:
# Setup ssh-key for user if not exists
ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

/home/pi/.ssh/id_rsa


Den SSH Zugang vorbereiten. Zunächst aus `knwon_hosts` löschen. Dann wieder bekannt geben und den ssh-key übertragen.

In [8]:
ping -c 1 cluster-01.local

PING cluster-01.local (172.24.2.2) 56(84) bytes of data.
64 bytes from 172.24.2.2: icmp_seq=1 ttl=64 time=0.857 ms

--- cluster-01.local ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.857/0.857/0.857/0.000 ms


In [9]:
ping -c 1 cluster-01.local | grep PING | cut -d '(' -f 2 | cut -d')' -f 1

172.24.2.2


In [10]:
for i in 01 02 03 04; do
    # Hostname only
    CL_HOSTNAME=cluster-${i}
    # Get Ip via local Domain (avahi)
    CL_IP=$(ping -c 1 ${CL_HOSTNAME}.local | grep PING | cut -d '(' -f 2 | cut -d')' -f 1)
    # Cluster Hostname only
    ssh-keygen -R ${CL_HOSTNAME}
    # Cluster with Domain local
    ssh-keygen -R ${CL_HOSTNAME}.local
    # IP Cluster 
    ssh-keygen -R ${CL_IP}
    # Add Cluster into known_hosts
    ssh-keyscan -H ${CL_HOSTNAME}.local >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_HOSTNAME} >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_IP} >> ~/.ssh/known_hosts
    # SSH key copy into Cluster
    sshpass -p raspberry ssh-copy-id pi@${CL_HOSTNAME}.local
done

/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host cluster-01.local found: line 13 type RSA
# Host cluster-01.local found: line 14 type ECDSA
# Host cluster-01.local found: line 15 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host 172.24.2.2 found: line 13 type ED25519
# Host 172.24.2.2 found: line 14 type RSA
# Host 172.24.2.2 found: line 15 type ECDSA
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
getaddrinfo cluster-01: Name or service not known
# 172.24.2.2 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 172.24.2.2 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 172.24.2.2 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
/usr/bin/ssh-copy-id: INFO: attempting to log i

## Dem Swarm beitreten

In [11]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
dchzaekl2ka9up3i2fez7lmiu *   cluster-00          Ready               Active              Leader


Alle Clients als Worker dem Swarm beitreten.

In [12]:
for i in 01 02 03 04; do
    CL_HOSTNAME=cluster-${i}.local
    echo $CL_HOSTNAME will join into swarm
    CL_IP=$(ping -c 1 ${CL_HOSTNAME} | grep PING | cut -d '(' -f 2 | cut -d')' -f 1)
    echo $CL_IP
    ssh ${CL_HOSTNAME} "docker swarm join --token $JOIN_TOKEN --advertise-addr ${CL_IP} ${IP_LEADER}:2377"
done

cluster-01.local will join into swarm
172.24.2.2
This node joined a swarm as a worker.
cluster-02.local will join into swarm
172.24.2.3
This node joined a swarm as a worker.
cluster-03.local will join into swarm
172.24.2.4
This node joined a swarm as a worker.
cluster-04.local will join into swarm
172.24.2.5
This node joined a swarm as a worker.


In [13]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
dchzaekl2ka9up3i2fez7lmiu *   cluster-00          Ready               Active              Leader
qixd9zpch292ptmo0nz5mamfz     cluster-01          Ready               Active              
un90wiszvf91ytqp7p6gr14rx     cluster-02          Ready               Active              
l5oq7336ndunmlmx5jdujpk11     cluster-03          Ready               Active              
svbz48dj6c6x0q69lmj0s92dv     cluster-04          Ready               Active              


## Visualizer anschauen

Einen graphischen Output als Service starten.

In [14]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  --no-resolve-image \
  --detach=false \
  alexellis2/visualizer-arm

au4wa30yvxgzptu1iuv46kadj

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
fy: Service converged to verify that tasks are stable... 

In [15]:
echo "http://${IP_LEADER_WLAN}:8000"

http://172.24.1.1:8000


In [16]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


Auf allen nodes den Monitor Dienst installieren. Dieser überwacht, wie viele whoami Container laufen.

Grün: Ein whoami-Container wird gestartet.

Rot: Ein whoami-Container stoppt.

Gelb: Für die Version 1.1.0

Blau: Für die Version 1.2.0

In [17]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  --no-resolve-image \
  --detach=false \
  stefanscherer/monitor:1.1.0

9qc43wo7414wjn5j9o4o3jywp

all progress: 0 out of 5 tasks 
7336ndun: pending   
9zpch292: pending   
wiszvf91: pending   
48dj6c6x: pending   
all progress: 5 out of 5 tasks 
fy: Service converged to verify that tasks are stable... 

Es läuft noch kein whoami.

In [18]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
9qc43wo7414w        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [19]:
docker service create --name whoami \
  --no-resolve-image \
  --detach=false \
  stefanscherer/whoami:1.1.0 

qqdf073o5khn8q6xtz9eotw6v

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
fy: Service converged to verify that tasks are stable... 

In [20]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
9qc43wo7414w        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
qqdf073o5khn        whoami              replicated          1/1                 stefanscherer/whoami:1.1.0         


In [21]:
docker service scale --detach=false whoami=5

whoami scaled to 5

all progress: 0 out of 5 tasks 
   K$<3>$<3>
   K$<3>$<3>
   K$<3>$<3>
   K$<3>$<3>
all progress: 5 out of 5 tasks 
fy: Service converged to verify that tasks are stable... 

In [22]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
9qc43wo7414w        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
qqdf073o5khn        whoami              replicated          5/5                 stefanscherer/whoami:1.1.0         


In [23]:
docker service scale --detach=false whoami=15

whoami scaled to 15

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
fy: Service converged to verify that tasks are stable... 

In [24]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
9qc43wo7414w        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
qqdf073o5khn        whoami              replicated          15/15               stefanscherer/whoami:1.1.0         


In [25]:
docker service scale --detach=false whoami=40

whoami scaled to 40

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [26]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

image stefanscherer/whoami:1.2.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.2.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [27]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
9qc43wo7414w        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
qqdf073o5khn        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [28]:
docker service scale --detach=false whoami=1

whoami scaled to 1

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks  
fy: Service converged to verify that tasks are stable... 

In [29]:
docker service scale --detach=false whoami=40

whoami scaled to 40

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [30]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
9qc43wo7414w        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
qqdf073o5khn        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [31]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.1.0 whoami

image stefanscherer/whoami:1.1.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.1.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [32]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

image stefanscherer/whoami:1.2.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.2.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [33]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
9qc43wo7414w        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
qqdf073o5khn        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [34]:
END=10
for i in $(seq 1 $END); do
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    docker service scale --detach=false whoami=1 
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
    docker service scale --detach=false whoami=15
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
    docker service scale --detach=false whoami=40
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
done

image stefanscherer/whoami:1.1.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.1.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 40 out of 40 tasks 
whoami scaled to 1ged to verify that tasks are stable... 

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks  
image stefanscherer/whoami:1.2.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.2.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
whoami scaled to 15ed to verify that tasks are stable... 

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of

:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoami scaled to 40ed to verify that tasks are stable... 

all progress: 40 out of 40 tasks 
image stefanscherer/whoami:1.1.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.1.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 40 out of 40 tasks 
image stefanscherer/whoami:1.2.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.2.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 40 out of 40 tasks 
image stefanscherer/whoami:1.1.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.1.0 independently,
possibly leading to different nodes running different
versions of

possibly leading to different nodes running different
versions of the image.

whoami

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
image stefanscherer/whoami:1.2.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.2.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoami scaled to 40ed to verify that tasks are stable... 

all progress: 40 out of 40 tasks 
image stefanscherer/whoami:1.1.0 could not be accessed on a registry to record
its d

its digest. Each node will access stefanscherer/whoami:1.2.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
whoami scaled to 15ed to verify that tasks are stable... 

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
image stefanscherer/whoami:1.1.0 could not be accessed on a registry to record
its digest. Each node will access stefanscherer/whoami:1.1.0 independently,
possibly leading to different nodes running different
versions of the image.

whoami

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress

# Aufräumen

In [35]:
docker service rm whoami

whoami


In [36]:
docker service rm monitor

monitor


In [37]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
dchzaekl2ka9up3i2fez7lmiu *   cluster-00          Ready               Active              Leader
qixd9zpch292ptmo0nz5mamfz     cluster-01          Ready               Active              
un90wiszvf91ytqp7p6gr14rx     cluster-02          Ready               Active              
l5oq7336ndunmlmx5jdujpk11     cluster-03          Ready               Active              
svbz48dj6c6x0q69lmj0s92dv     cluster-04          Ready               Active              


In [38]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
au4wa30yvxgz        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [39]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i}.local "docker swarm leave --force"
done

Node left the swarm.
Node left the swarm.
Node left the swarm.
Node left the swarm.


In [40]:
docker service rm visualizer

visualizer


In [41]:
docker swarm leave --force

Node left the swarm.
